# Images and Coordinates in Astronomy

In this lesson we are going to look at aspects of processing and viewing images specific to Astronomy and Solar Astronomy. By the end of this lesson you should understand:

* Projected Coordinate Systems in Images
* World Coordinate Systems
* Using WCS to calculate coordinates in images
* Plotting images with WCS in images
* Using SunPy Map

## Projected Coordinate Systems

When taking images of the sky, we are projecting the spherical celestial coordinate system onto a 2-dimensional plane, which means that there is no simple linear relation between pixel coordinates and celestial coordinates

There are multiple coordinate systems used to describe the locations in 2D and 3D space for both Astronomy and Solar Physics. We shall use a couple of these systems here as examples but if you want to know more about them there are many of resources avalible.

![Spherical Coordinates](coord_inset.png)


### Physical Coordinate Systems

Projected coordinate systems are one type of physical coordinate systems, they are the one we will be focusing on in this lesson due to their applicability to imaging data. Astropy and SunPy support representing point in many different physical coordinate systems, both projected and fully 3D, such as ICRS or Helioprojective.

In [1]:
import astropy.units as u
import sunpy.coordinates
from astropy.coordinates import SkyCoord

In [6]:
icrs = SkyCoord(10*u.deg, 86*u.deg, frame="icrs")
icrs

<SkyCoord (ICRS): (ra, dec) in deg
    (10., 86.)>

In [7]:
icrs.transform_to("galactic")

<SkyCoord (Galactic): (l, b) in deg
    (122.7151072, 23.13306851)>

In [9]:
hpc = SkyCoord(100*u.arcsec, 700*u.arcsec, obstime="now", frame="helioprojective")
hpc

<SkyCoord (Helioprojective: obstime=2018-09-07 09:52:21.588846, rsun=695508.0 km, observer=<HeliographicStonyhurst Coordinate (obstime=2018-09-07 09:52:21.588846): (lon, lat, radius) in (deg, deg, AU)
    (0., 7.25031415, 1.00778173)>): (Tx, Ty) in arcsec
    (100., 700.)>

In [10]:
hpc.transform_to("heliographic_stonyhurst")

<SkyCoord (HeliographicStonyhurst: obstime=2018-09-07 09:52:21.588846): (lon, lat, radius) in (deg, deg, km)
    (10.34721556, 54.31823272, 695508.00000359)>

In [11]:
hpc.transform_to("icrs")

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, km)
    (56.31671893, 44.3053279, 1232623.89642392)>

### World Coordinate System

#### From pixels  to physical coordinates

The FITS files have a standard for describing the physical coordinate system associated with imaging data, this is called the world coordinate system or WCS, sometimes the specific FITS version of this is referred to as FITS-WCS.

There are multiple papers describing the FITS-WCS standard for various types of data, there is a list here: http://fits.gsfc.nasa.gov/fits_wcs.html

As you learned in the previous lesson we can load FITS files with Astropy. To demonstrate a simple example of a FITS file with FITS-WCS information in the header we shall use an image from SunPy:

In [12]:
from sunpy.data.sample import AIA_171_IMAGE
from astropy.io import fits



As you can see there are lots of keys in this and most other real world FITS headers. The ones we need to understand for FITS-WCS are:

Reference Pixel and Coordinate:

In [13]:
AIA_171_IMAGE

'C:\\Users\\mkb18105\\sunpy\\data/sample_data\\AIA20110607_063302_0171_lowres.fits'

Pixel resolution (at the reference pixel):

In [14]:
hdulist = fits.open(AIA_171_IMAGE)
hdulist.verify('silentfix')
hdulist[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1024                                                  
NAXIS2  =                 1024                                                  
BLD_VERS= 'V5R12X  '                                                            
LVL_NUM =                  1.5                                                  
T_REC   = '2011-06-07T06:33:03Z'                                                
TRECSTEP=                  1.0                                                  
TRECEPOC= '1977.01.01_00:00:00_TAI'                                             
TRECROUN=                    1                                                  
ORIGIN  = 'SDO     '                                                            
DATE    = '2012-10-16T19:45:

Rotation angle, in degress (at the reference pixel):

Coordinate System and Projection:


<section class="challange panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Keyword Extraction</h2>
</div>


<div class="panel-body">

<p>Extract and print out the <code>TELESCOP</code> value from the header.</p>
<p>Next, extract the <code>WAVELNTH</code> and <code>WAVEUNIT</code> values, use these to construct an astropy Quantity object for the wavelength of this image.</p>

</div>

</section>



<section class="solution panel panel-primary">
<div class="panel-heading">
<h2><span class="fa fa-eye"></span> Solution</h2>
</div>

</section>


We could now sit down and work out how to convert from a pixel coordinate to a physical coordinate described by this header (Helioprojective).

However, we can cheat and just use Astropy.

We can convert from pixel to world coordinate:

Or back again:

The last parameter to the two above examples is the 'origin' parameter. It is a flag that tells WCS if you indexes should be 0-based (like numpy) or 1-based (like FITS).
Here we are using 0 as we want to convert to and from numpy indexes of the array.

## Where are my Pixels?

```
[-500, 0]
[500, 500]
[0, 0]
```


<section class="solution panel panel-primary">
<div class="panel-heading">
<h2><span class="fa fa-eye"></span> Solution</h2>
</div>

</section>


### WCS and SkyCoord

A common usecase for WCS + Coordinates is to store or transform from pixel coordinates to one or more different physical coordinates. Combining Astropy WCS and Coordinates makes this easy.

Assuming we have the WCS object we created from the FITS header above we can get an astropy Coordinate Frame:

We can then use this when creating `SkyCoord` objects:

*Note: For solar data you should always use `sunpy.map` for this*

## Coordinate Aware Plotting

In this section we are going to use the `astropy.visualization.wcsaxes` subpackage to make WCS aware image plots.

For this example we are going to use a Hubble image.

This image now has physcial labels in the native coordinate system of the image. We can see what the coordinate system and projection of this image is using the 'CTYPE' header entries we saw earlier.

We can tell that this is in the FK5 coordinate system by the presence of a 'equinox' entry in the header:


<section class="challange panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Add some labels</h2>
</div>


<div class="panel-body">

<p>Now we have a nice plot with physically meaningful ticks, we should label our axes.</p>
<p>Add labels to the axes saying "Right Ascension [degrees]" and "Declination [degrees]"</p>
<p>Also overlay a coordinate grid using:
<code>ax.coords.grid()</code>
Look up the documentation for this method to see what parameters you can specify.</p>

</div>

</section>



<section class="solution panel panel-primary">
<div class="panel-heading">
<h2><span class="fa fa-eye"></span> Solution</h2>
</div>

</section>


Now we have a nice plot, we can do a couple of things to plot.

### Overplotting in Pixel Coordinates

### Overplotting in World Coordinates


<section class="challange panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Overplot in a different coordinate system</h2>
</div>


<div class="panel-body">

<p>Using <code>ax.get_transform()</code> you can specify coordinates in any system that astropy coordinates can transform to, try overplotting a point in the <code>'galactic'</code> system.</p>
<p>Also overlay a coordinate grid using:
<code>ax.coords.grid()</code>
Look up the <a href="http://wcsaxes.readthedocs.io/en/latest/overlaying_coordinate_systems.html">documentation</a> for this method to see what parameters you can specify.</p>

</div>

</section>


###  Overplotting Another Coordinate System

## SunPy Map

The SunPy Map class is a wrapper for solar images which makes some of the above opertations easier.

### Coordinate Systems

The SunPy map will calculate some of the things we did earlier automatically, and add some extra information.

### Plotting

SunPy map uses these attributes to do it's plotting with WCSAxes like we did manually earlier. We can therefore use the overplotting and grid techniques to improve the plot:


<section class="challange panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Rotate your Owl</h2>
</div>


<div class="panel-body">

<p>Why is the Sun wonky?</p>
<p>Use the <a href="http://docs.sunpy.org/en/stable/code_ref/map.html#sunpy.map.mapbase.GenericMap.rotate"><code>rotate()</code></a> method of SunPy Map to align the coordinate grid to the pixel grid in this sample image.</p>
<p>Once you have run rotate, plot the resulting image, and compare with the one above.</p>

</div>

</section>
